In [ ]:
# -*- coding: utf-8 -*-
"""
PRONÓSTICO DIARIO A 1 AÑO CON EXÓGENA (TEMPERATURA) POR SUBÁREA
================================================================

Objetivo
--------
Pronosticar la demanda eléctrica DIARIA por subárea a 365 días (1 año) incorporando
**temperatura** como variable exógena, más calendario y estacionalidad anual.

Modelo
------
- **SARIMAX** con orden fijo y robusto para carga diaria:
  - No estacional: (0,1,1)  -> diferencia para tendencia + MA para autocorrelación corta
  - Estacional semanal: (0,1,1,7) -> diferencia semanal + MA estacional para patrón laboral/fin de semana
- **Exógenas**:
  - **Temperatura diaria** (avg) y **Temperatura²** (para capturar forma en U: frío y calor ↑ demanda)
  - **Dummies de día-de-semana** (dow)
  - **Festivo** (0/1)
  - **Fourier anual** (K armónicos) para estacionalidad larga sin 365 dummies

Validación
----------
- **Hold-out del último año**: entreno con años previos y pronostico 365 días.
- Métricas: **MAPE**, **RMSE** y baseline **naïve anual** (mismo día del año pasado).

Futuro
------
- Necesitamos **exógena futura** (temperatura). Aquí muestro un **escenario P50 climatológico**:
  media histórica de temperatura por (día-del-año).
- Si tienes un **pronóstico real** (CSV), ver función `load_future_temp_from_csv(...)`.

Adaptación a datos reales
-------------------------
- Reemplaza `simulate_daily_panel(...)` por tu dataframe con columnas:
  `date (D)`, `subarea (str)`, `demand (float)`, `temp (float)`, `holiday (0/1)`.
- Asegúrate de **no tener huecos** y que la malla sea diaria.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi

from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

# -------------------------
# 0) Parámetros generales
# -------------------------
RANDOM_SEED  = 42
np.random.seed(RANDOM_SEED)

N_SUBAREAS   = 6
START_DATE   = "2020-01-01"
END_DATE     = "2024-12-31"   # ~5 años diarios
TEST_DAYS    = 365            # último año para evaluar
FORECAST_H   = 365            # horizonte futuro (1 año)

FOURIER_PERIOD = 365.25
K_FOURIER      = 3            # nº de armónicos anuales (sube a 5 si necesitas más detalle)

# --------------------------------
# 1) Simulación (demanda+temperatura)
# --------------------------------
def simulate_temp_daily(idx, subarea_id):
    """
    Temperatura diaria (°C) simulada:
    - Componente anual (seno/coseno)
    - Sesgo pequeño por subárea
    - Ruido blanco
    """
    doy  = idx.dayofyear.values
    bias = (subarea_id - 3) * 0.3
    annual = 9*np.sin(2*np.pi*doy/365.25 - 0.6) + 4*np.cos(2*np.pi*2*doy/365.25)
    noise  = np.random.normal(0, 1.5, size=len(idx))
    return 18 + bias + annual + noise

def simulate_demand_daily(idx, subarea_id, temp):
    """
    Demanda diaria simulada:
    - Base por subárea
    - Patrón semanal: menor en sáb-dom
    - Estacionalidad anual suave
    - Efecto temperatura en U (lineal+cuadrático)
    - Festivos (reducción)
    - Tendencia leve
    - Ruido
    """
    dow = idx.dayofweek.values
    doy = idx.dayofyear.values
    t   = np.arange(len(idx))

    base   = 220 + 18*(subarea_id - 3)
    weekly = -18*(dow >= 5).astype(float)              # fin de semana
    annual = 8*np.cos(2*np.pi*doy/365.25) + 4*np.sin(2*np.pi*2*doy/365.25)
    trend  = 0.002 * t

    # Festivos fijos (ejemplo genérico)
    holidays = pd.Series(0, index=idx)
    for y in np.unique(idx.year):
        for m, d in [(1,1), (5,1), (12,25)]:
            ts = pd.Timestamp(y, m, d)
            if ts in set(idx):
                holidays[ts] = 1
    holidays = holidays.reindex(idx, fill_value=0).astype(int).values
    holiday_effect = -24 * holidays

    # Efecto temperatura: centro en 18°C (forma U): +a*(temp-18) + b*(temp-18)^2
    x = (temp - 18.0)
    temp_effect = 0.6 * x + 0.45 * (x**2)

    noise = np.random.normal(0, 5, size=len(idx))

    y = base + weekly + annual + trend + holiday_effect + temp_effect + noise
    return np.maximum(y, 10), holidays

def simulate_daily_panel():
    """
    DataFrame largo:
      date | subarea | demand | temp | holiday
    """
    idx = pd.date_range(START_DATE, END_DATE, freq="D")
    dfs = []
    for s in range(1, N_SUBAREAS+1):
        temp = simulate_temp_daily(idx, s)
        demand, holidays = simulate_demand_daily(idx, s, temp)
        df = pd.DataFrame({
            "date": idx,
            "subarea": f"S{s}",
            "demand": demand,
            "temp": temp,
            "holiday": holidays
        })
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

data = simulate_daily_panel().sort_values(["subarea", "date"]).reset_index(drop=True)

# --------------------------------
# 2) Features: Fourier + calendario + temperatura (exógena)
# --------------------------------
def fourier_terms(dates, period=FOURIER_PERIOD, K=K_FOURIER, prefix="yr"):
    """
    Términos de Fourier anuales: sin/cos para capturar estacionalidad larga
    sin crear 365 dummies; K controla la complejidad de la forma anual.
    """
    t = np.arange(len(dates))
    out = {}
    for k in range(1, K+1):
        out[f"{prefix}_sin_{k}"] = np.sin(2*np.pi*k*t/period)
        out[f"{prefix}_cos_{k}"] = np.cos(2*np.pi*k*t/period)
    return pd.DataFrame(out, index=dates)

def build_exog(df):
    """
    Exógenas para SARIMAX:
      - temp, temp^2
      - dummies de día-de-semana (dow_1..dow_6; base = lunes dow_0)
      - holiday
      - Fourier anual (K armónicos)
    """
    exog = pd.DataFrame(index=df.index)

    # Temperatura (lineal y cuadrática)
    exog["temp"] = df["temp"].values
    exog["temp2"] = (df["temp"].values - 18.0)**2  # centrada en 18 para estabilidad

    # Dummies de día-de-semana
    dummies = pd.get_dummies(df["date"].dt.dayofweek, prefix="dow", drop_first=True).astype(int)
    exog = pd.concat([exog, dummies], axis=1)

    # Festivo
    exog["holiday"] = df["holiday"].astype(int).values

    # Fourier anual
    ft = fourier_terms(df["date"], period=FOURIER_PERIOD, K=K_FOURIER, prefix="yr")
    ft.index = df.index
    exog = pd.concat([exog, ft], axis=1)

    return exog

def align_exog(train_df, test_df, future_df):
    """
    Asegura mismas columnas en TRAIN/TEST/FUTURE (p.ej., si falta algún dow en un segmento).
    """
    ex_tr = build_exog(train_df)
    ex_te = build_exog(test_df)
    ex_fu = build_exog(future_df)

    cols = sorted(set(ex_tr.columns) | set(ex_te.columns) | set(ex_fu.columns))
    ex_tr = ex_tr.reindex(columns=cols, fill_value=0)
    ex_te = ex_te.reindex(columns=cols, fill_value=0)
    ex_fu = ex_fu.reindex(columns=cols, fill_value=0)
    return ex_tr, ex_te, ex_fu

# --------------------------------
# 3) Split temporal: train/test
# --------------------------------
last_day   = data["date"].max()
test_start = last_day - pd.Timedelta(days=TEST_DAYS) + pd.Timedelta(days=1)

# --------------------------------
# 4) Modelo SARIMAX
# --------------------------------
def fit_sarimax_daily(y, exog, order=(0,1,1), seasonal_order=(0,1,1,7)):
    """
    SARIMAX robusto para carga diaria:
    - d=1 para tendencia, D=1 con s=7 para semana
    - MA y MA_seasonal capturan autocorrelación residual
    """
    model = SARIMAX(
        endog=y,
        exog=exog,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    res = model.fit(disp=False)
    return res

# --------------------------------
# 5) Escenarios de temperatura futura
# --------------------------------
def future_temp_scenario_climatology(df_sub, future_index):
    """
    Escenario P50 climatológico por (día-del-año): promedio histórico de temp para esa subárea.
    Úsalo cuando no tienes pronóstico meteorológico.
    """
    hist = df_sub.copy()
    hist["doy"] = hist["date"].dt.dayofyear
    clim = hist.groupby("doy")["temp"].mean().rename("temp").reset_index()

    fut = pd.DataFrame({"date": future_index})
    fut["doy"] = fut["date"].dt.dayofyear
    fut = fut.merge(clim, on="doy", how="left")
    fut["temp"] = fut["temp"].fillna(hist["temp"].mean())
    return fut["temp"].values

def load_future_temp_from_csv(csv_path, date_col="date", temp_col="temp"):
    """
    Carga un pronóstico de temperatura futura desde CSV con columnas:
    - date (YYYY-MM-DD)
    - temp (float)
    Devuelve un DataFrame con esas columnas para alinear por fecha.
    """
    df = pd.read_csv(csv_path, parse_dates=[date_col])
    df = df[[date_col, temp_col]].rename(columns={date_col: "date", temp_col: "temp"})
    return df

# --------------------------------
# 6) Entrenar / Validar / Pronosticar por subárea
# --------------------------------
results   = []
forecasts = []

for sub in data["subarea"].unique():
    df_sub = data[data["subarea"] == sub].copy().reset_index(drop=True)

    # Particiones
    df_train = df_sub[df_sub["date"] < test_start].copy()
    df_test  = df_sub[df_sub["date"] >= test_start].copy()   # último año

    # Exógenas consistentes
    ex_tr, ex_te, _ = align_exog(df_train, df_test, df_test)

    # Entrenar SARIMAX en TRAIN
    y_tr = df_train["demand"].values
    res  = fit_sarimax_daily(y_tr, ex_tr)

    # Pronóstico del año hold-out (TEST)
    yhat_test = res.predict(start=len(y_tr), end=len(y_tr)+len(df_test)-1, exog=ex_te)
    y_true    = df_test["demand"].values

    mape = mean_absolute_percentage_error(y_true, yhat_test) * 100
    rmse = np.sqrt(mean_squared_error(y_true, yhat_test))

    # Baseline naïve anual (mismo día del año pasado)
    if len(df_train) >= 365:
        naive_last_year = df_train["demand"].values[-365:][:len(df_test)]
        mape_naive = mean_absolute_percentage_error(y_true, naive_last_year) * 100
    else:
        mape_naive = np.nan

    results.append({"subarea": sub, "MAPE_%": mape, "RMSE": rmse, "MAPE_naive_%": mape_naive})

    # ============ Pronóstico FUTURO 365 días ============
    last_obs = df_sub["date"].max()
    future_idx = pd.date_range(last_obs + pd.Timedelta(days=1), periods=FORECAST_H, freq="D")

    # --- EXÓGENA FUTURA: TEMPERATURA ---
    # Opción A (por defecto): escenario climatológico P50 (DOY-mean)
    temp_future = future_temp_scenario_climatology(df_sub, future_idx)

    # (Opcional) Opción B: pronóstico real desde CSV (descomenta y asegura fechas coincidan)
    # df_temp_fc = load_future_temp_from_csv("mi_pronostico_temps.csv")   # <-- tu archivo
    # temp_map   = df_temp_fc.set_index("date")["temp"]
    # temp_future = pd.Series(index=future_idx, data=np.nan)
    # temp_future.loc[temp_map.index] = temp_map.values
    # temp_future = temp_future.fillna(method="ffill").fillna(method="bfill").values

    # Construir DataFrame futuro con temp y festivos
    df_future = pd.DataFrame({
        "date": future_idx,
        "subarea": sub,
        "temp": temp_future,
        "holiday": 0
    })
    # Festivos futuros simples (reemplaza por tu calendario oficial)
    for y in np.unique(df_future["date"].dt.year):
        for m, d in [(1,1), (5,1), (12,25)]:
            ts = pd.Timestamp(y, m, d)
            if ts in set(df_future["date"]):
                df_future.loc[df_future["date"] == ts, "holiday"] = 1

    # Alinear exógenas (TRAIN/TEST/FUTURE) y reentrenar con TODO el histórico antes del futuro
    ex_tr2, ex_te2, ex_fu = align_exog(df_train, df_test, df_future)
    ex_full = build_exog(df_sub).reindex(columns=ex_tr2.columns, fill_value=0)
    res_full = fit_sarimax_daily(df_sub["demand"].values, ex_full)

    # Pronóstico 365 días
    yhat_future = res_full.predict(start=len(df_sub), end=len(df_sub)+FORECAST_H-1, exog=ex_fu)

    forecasts.append(pd.DataFrame({
        "date": future_idx,
        "subarea": sub,
        "forecast": yhat_future
    }))

# --------------------------------
# 7) Resultados y ejemplo de salida
# --------------------------------
metrics_df = pd.DataFrame(results)
metrics_df.loc[len(metrics_df)] = {
    "subarea": "GLOBAL",
    "MAPE_%": metrics_df["MAPE_%"].mean(),
    "RMSE": metrics_df["RMSE"].mean(),
    "MAPE_naive_%": metrics_df["MAPE_naive_%"].mean()
}

print("\nMÉTRICAS HOLD-OUT (último año):")
print(metrics_df.round(3))

forecast_df = pd.concat(forecasts, ignore_index=True).sort_values(["subarea","date"])
print("\nHEAD del pronóstico futuro (primeras 10 filas):")
print(forecast_df.head(10))

# (Opcional) Guardar CSV
# forecast_df.to_csv("forecast_diario_1anio_con_exog_temp_por_subarea.csv", index=False)

# (Opcional) Gráfica rápida para una subárea
# sub_demo = "S1"
# df_sub = data[data["subarea"] == sub_demo].copy().reset_index(drop=True)
# df_train = df_sub[df_sub["date"] < test_start]
# df_test  = df_sub[df_sub["date"] >= test_start]
# ex_tr, ex_te, _ = align_exog(df_train, df_test, df_test)
# res = fit_sarimax_daily(df_train["demand"].values, ex_tr)
# yhat_test = res.predict(start=len(df_train), end=len(df_train)+len(df_test)-1, exog=ex_te)
# plt.figure(figsize=(12,4))
# plt.plot(df_test["date"], df_test["demand"], label="Real")
# plt.plot(df_test["date"], yhat_test, label="Pronóstico")
# plt.title(f"{sub_demo}: Validación último año")
# plt.xlabel("Fecha"); plt.ylabel("Demanda diaria"); plt.legend(); plt.tight_layout(); plt.show()
